# Задание 1

### _Задача:_

Оценить следующие 4 варианта вложения денег: приобретение акций, 
оформление депозита, приобретение облигаций, приобретение драгоценных 
металлов по критерию «надежность вложения денег». По результатам 
парных сравнений установлено, что 2-й вариант ненамного лучше 1-го и 3-го 
и намного лучше 4-го, 1-й вариант имеет одинаковую надежность с 3-им и 
ненамного лучше 4-го, преимущество 3го над 4-тым – между слабым и 
существенным.

Для данной задачи необходимо:

- построить мультипликативную матрицу парных сравнений МПС, при условии что эксперт выставляет оценки в соответствии с фундаментальной шкалой;

- для заданной МПС D выполнить расчет главного собственного числа и подчиненного ему главного вектора;

- найти веса альтернатив используя методы парных сравнений ЕМ, RGMM, AN и «линия» (рассмотреть все эталоны);

- рассчитать отношения согласованности CR, геометрический индекс согласованности GCI и гармонические отношения согласованности HCR;

- оценить согласованность МПС по спектральному коэффициенту согласованности приняв за Rh - спектр, построенный для множества оценок весов, полученных из матриц, порожденных из A методом «линия», h=1,…, n.

- оценить полученные результаты.


In [1]:
# импорт необходимых зависимостей
import numpy as np
import statistics
import math

In [2]:
# построение мультипликативной МПС
matrix = np.array([
        [1, 1/3, 1, 3],
        [3, 1, 3, 7],
        [1, 1/3, 1, 4],
        [1/3, 1/7, 1/4, 1]
    ])

#### Поиск весов альтернатив методом EM:

In [3]:
# получение собственных чисел и векторов исходной матрицы
l, v = np.linalg.eig(matrix)

In [4]:
# определение максимального (главного) собственного числа матрицы и избавление (по возможности) от мнимой части
lmax = np.max(l)
lmax = (lmax, lmax.real)[bool(lmax.imag == 0j)]
lmax

4.027401082293499

In [5]:
# определение главного собственного вектора матрицы, и соответственно вектора весов
vmax = v[np.where(l == lmax)]
vmax

array([[ 0.31131963+0.j        , -0.55517491+0.j        ,
        -0.07413203-0.10604736j, -0.07413203+0.10604736j]])

In [6]:
# вычисление индекса согласованности
n = np.shape(matrix)[0]
ci = (lmax - n) / (n - 1)
ci

0.00913369409783312

In [7]:
# вычисление отношения согласованности
a_mrci = 0.89 # константное значение для n = 4
cr = ci / a_mrci
cr

0.01026257763801474

#### Поиск весов альтернатив методом RGMM:

In [8]:
# поиск вектора средних геометрических строк матрицы
v_means = [statistics.geometric_mean(row) for row in matrix]
v_means

[1.0, 2.8173132472612576, 1.074569931823542, 0.3303164318013807]

In [9]:
# нормировка и получение вектора весов
v_means_sum = np.sum(v_means)
rgmm_weights = v_means / v_means_sum
rgmm_weights

array([0.19149019, 0.53948785, 0.2057696 , 0.06325236])

In [10]:
# вычисление геометрического индекса согласованности
errors_matrix = np.zeros_like(matrix)
for i in range(np.shape(matrix)[0]):
    for j in range(np.shape(matrix)[0]):
        errors_matrix[i][j] = matrix[i][j] * (v_means[i] / v_means[j])
errors_matrix = np.log2(errors_matrix)
gci_multiplier = np.sum(errors_matrix)
gci = (2 * gci_multiplier) / ((n - 1) * (n - 2))
gci

2.9605947323337506e-16

#### Поиск весов альтернатив методом AN:

In [11]:
# получение вектора весов
an_columns_sums = matrix.sum(axis=0)
an_weights = an_columns_sums ** -1
an_weights

array([0.1875    , 0.55263158, 0.19047619, 0.06666667])

In [12]:
# вычисление гармонического индекса согласованности
hm = n / np.sum(an_weights)
hci = ((hm - n) * (n + 1)) / (n * (n - 1))
hci

0.004555021518550066

In [13]:
# вычисление гармонического отношения согласованности
hcr = hci / a_mrci
hcr

0.005118001706236029

#### Поиск весов альтернатив методом "линия":

In [14]:
# инициализация и заполнения матриц, полученных перемножением i-й строки на все остальные, кроме самой себя
standard_matrix = []
for i in range(np.shape(matrix)[0]):
    addition = np.copy(matrix)
    addition = np.delete(addition, i, axis=0)
    addition *= matrix[i]
    addition = np.insert(addition, i, matrix[i], axis=0)
    standard_matrix.append(addition)
standard_matrix = np.array(standard_matrix)
standard_matrix

array([[[ 1.        ,  0.33333333,  1.        ,  3.        ],
        [ 3.        ,  0.33333333,  3.        , 21.        ],
        [ 1.        ,  0.11111111,  1.        , 12.        ],
        [ 0.33333333,  0.04761905,  0.25      ,  3.        ]],

       [[ 3.        ,  0.33333333,  3.        , 21.        ],
        [ 3.        ,  1.        ,  3.        ,  7.        ],
        [ 3.        ,  0.33333333,  3.        , 28.        ],
        [ 1.        ,  0.14285714,  0.75      ,  7.        ]],

       [[ 1.        ,  0.11111111,  1.        , 12.        ],
        [ 3.        ,  0.33333333,  3.        , 28.        ],
        [ 1.        ,  0.33333333,  1.        ,  4.        ],
        [ 0.33333333,  0.04761905,  0.25      ,  4.        ]],

       [[ 0.33333333,  0.04761905,  0.25      ,  3.        ],
        [ 1.        ,  0.14285714,  0.75      ,  7.        ],
        [ 0.33333333,  0.04761905,  0.25      ,  4.        ],
        [ 0.33333333,  0.14285714,  0.25      ,  1.        ]]])

In [15]:
# заполнение матрицы весов альтернатив найденных методом линия
standard_matrix_weights = []
for i in range(len(standard_matrix)):
    column_sum = standard_matrix[i].sum(axis=0)
    weights_addition = standard_matrix[i][i] / column_sum
    standard_matrix_weights.append(weights_addition)
standard_matrix_weights = np.array(standard_matrix_weights)
standard_matrix_weights

array([[0.1875    , 0.40384615, 0.19047619, 0.07692308],
       [0.3       , 0.55263158, 0.30769231, 0.11111111],
       [0.1875    , 0.40384615, 0.19047619, 0.08333333],
       [0.16666667, 0.375     , 0.16666667, 0.06666667]])

#### Оценка согласованности МПС по спектральному коэффициенту согласованности (спектр Rh - строка полученной ранее матрицы весов)

In [16]:
# вычисление масштабного коэффициента и средней оценки множества ЭО
k_n = np.shape(standard_matrix_weights)[0]
k_m = np.shape(standard_matrix_weights)[1]
g = k_m / (math.log(k_m) * k_n * math.log(k_n))
a = np.mean(standard_matrix_weights)

In [17]:
# вычисление спектральных коэффициентов согласованности для матрицы (i-й коэффициент соответствует i-й строке)
concordance_coefficients = []
for row in standard_matrix_weights:
    first_nom_part = 0
    second_nom_part = 0
    first_denominator_part = 0
    for i in range(k_n):
        first_nom_part += (row[i] * math.fabs(i - a))
        second_nom_part += ((row[i] / k_m) * math.log((row[i] / k_m)))
        first_denominator_part += math.fabs(i - (k_n + 1) / 2)
    first_nom_part /= k_m
    first_denominator_part *= g
    coefficient = 1 - ((first_nom_part - second_nom_part) / (first_denominator_part + math.log(k_n)))
    concordance_coefficients.append(coefficient)
concordance_coefficients = np.array(concordance_coefficients)
concordance_coefficients

array([0.79405418, 0.7240821 , 0.79177369, 0.81041681])

In [18]:
# определение спектрального коэффициента согласованности матрицы
minimal_concordance_coefficient = np.min(concordance_coefficients)
minimal_concordance_coefficient

0.724082104077029